In [1]:
#%%
from model import Generator, Discriminator, DomainClassifier
from utils import audio_process
import IPython.display as ipd
import tensorflow as tf
import numpy as np
import os

/home/halley/anaconda3/envs/tensorflow_2.1/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
#%%
'''
Usage:
  Interactive web page via jupyter
'''
ckpt_path = './ckpt_wgangp'

G = Generator()
dummy_wav = np.zeros((1, 36, 512, 1), dtype=np.float32)
dummy_speaker_onehot = np.zeros((1, 2), dtype=np.float32)
G(dummy_wav, dummy_speaker_onehot, True)
restored_ckpt = tf.train.Checkpoint(G=G)
latest_ckpt = tf.train.latest_checkpoint(ckpt_path)
restored_ckpt.restore(latest_ckpt).assert_existing_objects_matched()

In [5]:
#%%
from utils.mcep_normalizer import McepNormalizer

def pad_mcep(mcep, frame_size=512):
    length = mcep.shape[1]
    if length/frame_size != 1:
        pad_length = frame_size - length%frame_size
    padded_mcep = np.pad(mcep, ((0,0), (0,pad_length)))
    return padded_mcep

frame_size = 512

mcep_normalizer = McepNormalizer('./train/norm_dict.pkl')
result_dir = 'result'
input_wav = '../dataset/speakers/p2/p2_260.wav'
src_speaker = input_wav.split('/')[-1].split('_')[0][1:]
target_speaker = 2
speaker_counts = 2

f0, ap, sp, mcep = audio_process.wav2mcep(input_wav)
speaker = np.eye(speaker_counts, dtype=np.float32)[target_speaker-1]
speaker = speaker[np.newaxis, :]
mcep = pad_mcep(mcep, frame_size)
mcep = mcep.astype(np.float32)
tape = []
for idx in range(0, mcep.shape[1]-frame_size+1, frame_size):
    seg_mcep = mcep[np.newaxis, :, idx:idx+frame_size, np.newaxis]
    converted = G(seg_mcep, speaker, False)
    converted_np = converted.numpy()[0,:,:,0]
    converted_np = mcep_normalizer.mcep_denorm(converted_np, str(target_speaker))
    tape.append(converted_np)
tape = np.concatenate(tape, axis=1)[:, :sp.shape[0]]
tape = np.ascontiguousarray(tape.T, dtype=np.float64)
converted_f0 = mcep_normalizer.f0_convert(f0, src_speaker, str(target_speaker))
converted_wav = audio_process.mcep2wav(tape, converted_f0, ap)

filename = input_wav.split('/')[-1].split('.')[0]
filename = filename + '_to_p%d.wav' % (target_speaker)
filepath = os.path.join(ckpt_path, result_dir, filename)
os.makedirs(os.path.dirname(filepath), exist_ok=True)
abs_filepath = audio_process.write_wav(converted_wav, filepath)
ipd.Audio(abs_filepath)

src: 2 tgr:  2
